## Imports

In [12]:
import pandas as pd
import numpy as np

## Load / merge CSVs

In [10]:
detalle_2022 = pd.read_csv('ListaCobroDetalle2022.csv', dtype={0: str, 1: str, 2: str, 3: str, 8:str})
detalle_2023 = pd.read_csv('ListaCobroDetalle2023.csv', dtype={0: str, 1: str, 2: str, 3: str, 8:str})
detalle_2024 = pd.read_csv('ListaCobroDetalle2024.csv', dtype={0: str, 1: str, 2: str, 3: str, 8:str})
detalle_2025 = pd.read_csv('ListaCobroDetalle2025.csv', dtype={0: str, 1: str, 2: str, 3: str, 8:str})

df_detalle = pd.concat([detalle_2022, detalle_2023, detalle_2024, detalle_2025], ignore_index=True)

df_detalle.to_csv("df_Detalle.csv", index=False)
df_detalle.head()

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco
0,100613,83021,27169966,2,572.24,572.24,0.0,NaN,04
1,100613,87227,27169937,2,517.74,517.74,0.0,NaN,04
2,100613,103785,27169946,2,681.23,681.23,0.0,NaN,04
3,100613,110463,27169969,2,572.24,572.24,0.0,NaN,04
4,100613,113755,27169948,2,340.62,340.62,0.0,NaN,04


In [16]:
df_detalle['idRespuestaBanco'].unique()

array(['04', 'DD00031', '00', 'DD00028', '08', '01', 'INC9', nan, '02',
       'DD00042', 'DD00021', 'INC16', 'DD00033', '26', '03', '30',
       'DD00020', '83', '53', '19', 'DD00041', '13', '10', '88', '36',
       '42', '40', '18', '37', '06', '84'], dtype=object)

In [17]:
# Hacer el merge por la columna 'idListaCobro'
df_cobro = pd.read_csv("ListaCobro.csv", encoding='utf-8', dtype={0:str, 2:str})

df_ListaCobro = df_detalle.merge(
    df_cobro[['idListaCobro', 'fechaEnvioCobro']],
    on='idListaCobro',
    how='left'
)

df_ListaCobro.head()

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaEnvioCobro
0,100613,83021,27169966,2,572.24,572.24,0.0,NaN,04,30/09/2022
1,100613,87227,27169937,2,517.74,517.74,0.0,NaN,04,30/09/2022
2,100613,103785,27169946,2,681.23,681.23,0.0,NaN,04,30/09/2022
3,100613,110463,27169969,2,572.24,572.24,0.0,NaN,04,30/09/2022
4,100613,113755,27169948,2,340.62,340.62,0.0,NaN,04,30/09/2022


In [18]:
df_ListaCobro.to_csv("df_ListaCobro.csv", index=False, encoding='utf-8')

In [19]:
df_ListaCobro = pd.read_csv("df_ListaCobro.csv", encoding='utf-8', dtype={0: str, 1: str, 2: str, 3: str, 8:str})

In [20]:
# Merge de la nueva db
df_nuevo = pd.read_csv("Capital_Pagare_FechaAperturaCredito.csv", encoding='utf-8', dtype={0:str})

df_nuevo.columns = df_nuevo.columns.str.strip()

# Realizar el merge usando 'Crédito' y 'idCredito' como llave
df_merged = df_ListaCobro.merge(
    df_nuevo,
    left_on='idCredito',
    right_on='idcredito',
    how='left'
)

df_merged = df_merged.drop(columns=['idcredito'])

df_merged.to_csv("df_Merged.csv", index=False)
df_merged.head()

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaEnvioCobro,pagare,capital,fechaAperturaCredito
0,100613,83021,27169966,2,572.24,572.24,0.0,NaN,04,30/09/2022,44634.33,21000.0,30/12/2013
1,100613,87227,27169937,2,517.74,517.74,0.0,NaN,04,30/09/2022,40383.33,19000.0,12/2/2014
2,100613,103785,27169946,2,681.23,681.23,0.0,NaN,04,30/09/2022,53135.94,25000.0,19/02/2014
3,100613,110463,27169969,2,572.24,572.24,0.0,NaN,04,30/09/2022,44634.33,21000.0,26/02/2014
4,100613,113755,27169948,2,340.62,340.62,0.0,NaN,04,30/09/2022,26567.97,12500.0,21/03/2014


In [21]:
df_merged = pd.read_csv("df_Merged.csv", encoding='utf-8', dtype={0: str, 1: str, 2: str, 3: str, 8:str})

In [22]:
# Mascara cuando 'idRespuestaBanco' es nulo, y montocobrado es 0, entonces esos nulos reemplazarlos con '04'

mascara = (df_merged['idRespuestaBanco'].isnull()) & (df_merged['montoCobrado'] == 0)

df_merged.loc[mascara, 'idRespuestaBanco'] = '04'

print("Número de valores nulos restantes en idRespuestaBanco:", df_merged['idRespuestaBanco'].isnull().sum())

Número de valores nulos restantes en idRespuestaBanco: 84157


In [23]:
# Mascara cuando fila 'montoCobrar' y 'montoCobrado' son iguales, entonces reemplazar los nulos de'idRespuestaBanco' con '00'

# Crear la mascara
mascara = (df_merged['montoCobrar'] == df_merged['montoCobrado']) & (df_merged['idRespuestaBanco'].isnull())

df_merged.loc[mascara, 'idRespuestaBanco'] = '00'

print("Número de valores nulos restantes en idRespuestaBanco:", df_merged['idRespuestaBanco'].isnull().sum())

Número de valores nulos restantes en idRespuestaBanco: 50


In [24]:
df_mascara = df_merged

df_mascara.to_csv("df_Mascara.csv", index=False)

In [26]:
df_mascara = pd.read_csv('df_Mascara.csv', encoding='utf-8', dtype={0: str, 1: str, 2: str, 3: str, 8:str})

In [28]:
# Agregar columnas con nombre

from diccionarios import dict_bancos, dict_emisora, dict_emisora_TE, dict_respuesta_banco

df_mascara['idBancoStr'] = df_mascara['idBanco'].map(dict_bancos)
df_mascara['idRespuestaBancoStr'] = df_mascara['idRespuestaBanco'].map(dict_respuesta_banco)

In [29]:
df_diccionarios = df_mascara
df_diccionarios.to_csv("df_Diccionarios.csv", index=False, encoding='utf-8')
df_diccionarios.head()

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaEnvioCobro,pagare,capital,fechaAperturaCredito,idBancoStr,idRespuestaBancoStr
0,100613,83021,27169966,2,572.24,572.24,0.0,NaN,04,30/09/2022,44634.33,21000.0,30/12/2013,NaN,Cuenta Insuficiencia Fondos
1,100613,87227,27169937,2,517.74,517.74,0.0,NaN,04,30/09/2022,40383.33,19000.0,12/2/2014,NaN,Cuenta Insuficiencia Fondos
2,100613,103785,27169946,2,681.23,681.23,0.0,NaN,04,30/09/2022,53135.94,25000.0,19/02/2014,NaN,Cuenta Insuficiencia Fondos
3,100613,110463,27169969,2,572.24,572.24,0.0,NaN,04,30/09/2022,44634.33,21000.0,26/02/2014,NaN,Cuenta Insuficiencia Fondos
4,100613,113755,27169948,2,340.62,340.62,0.0,NaN,04,30/09/2022,26567.97,12500.0,21/03/2014,NaN,Cuenta Insuficiencia Fondos


In [30]:
df = pd.read_csv('df_Diccionarios.csv', dtype={0: str, 1: str, 2: str, 3: str, 8:str, 13:str, 14:str})

In [31]:
# Despues de 2021

df['fechaAperturaCredito'] = pd.to_datetime(df['fechaAperturaCredito'], dayfirst=True, errors='coerce')

df_filtrado = df[df['fechaAperturaCredito'] >= '2021-01-01']

In [32]:
df_filtrado['fechaAperturaCredito'] = pd.to_datetime(df_filtrado['fechaAperturaCredito'], dayfirst=True, errors='coerce')

años_unicos = df_filtrado['fechaAperturaCredito'].dt.year.dropna().unique()
años_unicos.sort()
print(años_unicos)

[2021 2022 2023 2024 2025]


/var/folders/lv/34x2z89s3s3bys5fzpv8k45c0000gn/T/ipykernel_10846/1042475209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['fechaAperturaCredito'] = pd.to_datetime(df_filtrado['fechaAperturaCredito'], dayfirst=True, errors='coerce')


In [33]:
df_filtrado.to_csv("df_2021_LIMPIO.csv", index=False)

In [36]:
df_filtrado.head()

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaEnvioCobro,pagare,capital,fechaAperturaCredito,idBancoStr,idRespuestaBancoStr
41,101188,599055,27240538,2,1765.53,1765.53,0.0,NaN,04,07/10/2022,183615.12,70000.00,2021-08-13,NaN,Cuenta Insuficiencia Fondos
139,100551,579674,27163668,21,506.38,506.38,0.0,NaN,DD00031,29/09/2022,48612.48,18204.19,2021-02-09,NaN,Cuenta con Insuficiencia de Fondos
140,100551,581020,27163709,21,4899.99,4899.99,0.0,NaN,DD00031,29/09/2022,470399.04,176153.00,2021-02-23,NaN,Cuenta con Insuficiencia de Fondos
141,100551,581980,27163662,127,444.82,444.82,0.0,NaN,DD00031,29/09/2022,16013.52,10000.00,2021-03-06,NaN,Cuenta con Insuficiencia de Fondos
142,100551,582409,27163723,21,847.85,847.85,0.0,NaN,DD00031,29/09/2022,81393.60,30480.00,2021-03-11,NaN,Cuenta con Insuficiencia de Fondos
